# TEST OF THE USER INTERFACE

## Basic initialization of the lorentzian manifold

Initialization of the Schwarzschild manifold $\mathcal M_{Schwarzschild}$ atlas with Schwarzschild chart (SD) and of the metric tensor $g$ :

In [22]:
%display latex

M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
#BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") #boyer-lindquist chart definition
SD.<t, r, th, ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi:periodic")

var('m', domain='real')
#var('m, a', domain='real') #definition of Kerr metric parameters

g = M.metric() #we put the label g on the metric tensor

#rho2 = r^2 + (a*cos(th))^2
#Delta = r^2 -2*m*r + a^2

#g[0,0] = -(1-2*m*r/rho2)
#g[0,3] = -2*a*m*r*sin(th)^2/rho2
#g[1,1], g[2,2] = rho2/Delta, rho2
#g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g[0, 0] = - (1 - 2*m/r)
g[1, 1] = 1/(1 - 2*m/r)
g[2, 2] = r^2
g[3, 3] = r^2*sin(th)^2

g.display()

g = (2*m/r - 1) dt*dt - 1/(2*m/r - 1) dr*dr + r^2 dth*dth + r^2*sin(th)^2 dph*dph

## Initialization of the orthonormal tetrad

Definition of the orthonormal tetrad frame $e_{(a)}$ (Chandrasekar notation) and its dual frame $e^{(a)} = \eta^{(a)(b)}g(e_{(b)},-)$ :

In [23]:
e0 = M.vector_field(1/sqrt(1-2*m/r), 0, 0, 0, frame=SD.frame(), name='e_{(0)}')
e1 = M.vector_field(0, sqrt(1-2*m/r), 0, 0, frame=SD.frame(), name='e_{(1)}')
e2 = M.vector_field(0, 0, 1/r, 0, frame=SD.frame(), name='e_{(2)}')
e3 = M.vector_field(0, 0, 0, 1/(r*sin(th)), frame=SD.frame(), name='e_{(3)}')


e = M.vector_frame('e', (e0, e1, e2, e3)) #costruction of the tetrad collection object e_{a}
de = e.coframe() #construction of the tetrad coframe object e^{a}  

## Inizialization of the ST bundle

Definition of the ST bundle:

In [24]:
from ST_bundle.ST_bundle import ST_bundle

In [25]:
STb = ST_bundle(M,e)

## Construction of scalar and spin_tensor objects and visualization

To build a collection of scalar fields $S_{(a) \ \ \ (b)}^{ \ \  \ (c)}$ or spinor tensors (1,2) $T_{(a)(b) \ \ (d)}^{ \quad (c)}$ on M with tindices (the objects list is generated by recursion):

In [26]:
S = STb.scalar(["down","up","down"],M.scalar_field({SD : 3}))
T = STb.spin_tensor(["down","down","up","down"],(1,2),STb.sbundle.section_module().tensor((1,2))) 

__repr__() method tells the tindices structure of the object, the type of the object (scalar, spinor tensor), the sindices if we have a tensor, and the frames associated:

In [27]:
S

['down', 'up', 'down'] Tetrad index family of scalar fields on M with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

In [28]:
T

['down', 'down', 'up', 'down'] Tetrad index family of Tensors (1, 2) on the spin bundle section module with respect to the tetrad frame Vector frame (M, (e_0,e_1,e_2,e_3))

Through the dunder methods getitem and setitem it is possible to access the object with specific indices:

In [29]:
S[0,1,2]

Scalar field on the 4-dimensional Lorentzian manifold M

In [30]:
S[0,1,2] = S[0,0,0] #error,if the object is immutable, see below

The display() method shows me the components of the object:

In [31]:
S.display()

[[[3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3]],
 [[3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3]],
 [[3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3]],
 [[3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3], [3, 3, 3, 3]]]

For tindices scalars, the compatibility with the Sage class Components is guaranteed. For example if I have a vector field $V$ on the tangent bundle, I can pass its components $V^{(a)}$ with respect to the tetrad frame $e_{(a)}$ to build a scalar with tindices:

In [32]:
V = M.vector_field([1,0,0,0])
S_from_comp = STb.set_scalar_from_Components(["up"],V.comp(e))
S_from_comp.display()

[sqrt(-2*m + r)/sqrt(r), 0, 0, 0]

## Object implemented in the package

Definition of $\eta^{(a)(b)}$ :

In [33]:
eta = STb.eta("up") #eta("down") for the lower version
eta.display()

[[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]]

Calculation of the Ricci rotation coefficents $\omega^{(a)}_{\ \ \ (b)(c)}$ :

In [34]:
Ric_coef = STb.Ricci_rotation_coefficents(["up","down","down"]) #You can choose the up down position of the three indices as you want
Ric_coef.display()

[[[0, 0, 0, 0],
  [m/(sqrt(-2*m + r)*r^(3/2)), 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, 0, 0]],
 [[m/(sqrt(-2*m + r)*r^(3/2)), 0, 0, 0],
  [0, 0, 0, 0],
  [0, 0, -sqrt(-2*m + r)/r^(3/2), 0],
  [0, 0, 0, -sqrt(-2*m + r)/r^(3/2)]],
 [[0, 0, 0, 0],
  [0, 0, sqrt(-2*m + r)/r^(3/2), 0],
  [0, 0, 0, 0],
  [0, 0, 0, -cos(th)/(r*sin(th))]],
 [[0, 0, 0, 0],
  [0, 0, 0, sqrt(-2*m + r)/r^(3/2)],
  [0, 0, 0, cos(th)/(r*sin(th))],
  [0, 0, 0, 0]]]

Definition of the identity matrix $\mathbb{1}$ :

In [35]:
Id = STb.Identity([]) #it is possible to create a multi tindices version
Id.display()

[1 0 0 0]
[0 1 0 0]
[0 0 1 0]
[0 0 0 1]

Definition of Dirac's gamma matrices $\{\gamma^{(a)}\}_{(a)=0,1,2,3}$ :

In [36]:
Gamma = STb.Dirac_matrices("up")
Gamma.display()

[
[ I  0  0  0]  [ 0  0  0  I]  [ 0  0  0  1]  [ 0  0  I  0]
[ 0  I  0  0]  [ 0  0  I  0]  [ 0  0 -1  0]  [ 0  0  0 -I]
[ 0  0 -I  0]  [ 0 -I  0  0]  [ 0 -1  0  0]  [-I  0  0  0]
[ 0  0  0 -I], [-I  0  0  0], [ 1  0  0  0], [ 0  I  0  0]
]

Check of the Clifford algebra $\{\gamma^{(a)},\gamma^{(b)}\} = 2\eta^{(a)(b)}\mathbb{1}$ :

In [ ]:
Gamma.anticomm(Gamma) == 2*eta@Id

Definition of the null matrix $0$ :

In [ ]:
Zero = STb.Zero([]) #it is possible to create a multi tindices version
Zero.display()

Calculation of the generators $\{\sigma^{(a)(b)}= \frac
{i}{2}[\gamma^{(a)},\gamma^{(b)}]\}_{(a),(b)=0,1,2,3}$ of the spinor representation of the Lorentz Group:

In [ ]:
sigma = STb.lgroup_spinor_generators("up")
sigma.display()

Calculation of spinor connection coefficents $(\Gamma_{(a)})^{i}_{ \ \ j}$ :

In [ ]:
Spinor_coef = STb.spinor_coef
Spinor_coef.display()

## Useful methods

With the method copy() we can create a deepcopy of the object:

In [ ]:
Gamma_copy = Gamma.copy()
print(Gamma_copy is Gamma)
print(Gamma_copy[0] is Gamma [0])

With the method set_immutable() we can set an object and its components immutable at all levels:

In [ ]:
Gamma_copy.set_immutable()
Gamma_copy[0] = Gamma[1] #error
Gamma_copy[0][0,0] = 3 #error, immutability at all levels

With the method get_tangent_tensor() we convert a tindices object into a collection of tensors on the tangent bundle: $T^{(a)\dots}_{(b)\dots} \to T^{\mu \dots}_{\nu \dots}$

For a scalar with tindices: $\omega^{(a)}_{ \ \ \ (b)(c)} -> \omega^{\alpha}_{ \ \ \ \beta\rho}$

In [ ]:
Ric_coef.get_tangent_tensor().display()

For a spinor tensor with tindices: $\gamma^{(a)} \to \gamma^{\mu}$

In [ ]:
Gamma.get_tangent_tensor()[1][2].display()